In [5]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver import ActionChains
import time

# Chromedriver 옵션 설정
options = webdriver.ChromeOptions()
# options.add_argument("headless")
    
# execute driver & connect page
driver = webdriver.Chrome('./chromedriver', options=options)
driver.get('http://www.statiz.co.kr/stat.php?re=1&lr=0')
time.sleep(3)

# 스크롤을 위한 ActionChains 모듈 선언
action = ActionChains(driver)

# 100개씩 정렬하기 위한 옵션설정
optionbutton = driver.find_element_by_xpath('/html/body/div[1]/div[1]/div/section[2]/div/div[2]/div[1]/div/div[8]/button')
driver.execute_script("window.scrollTo(0, 100);") # 광고로 인해 옵션버튼이 가려질 수 있으므로
time.sleep(2)
optionbutton.click()

# 정렬 개수 100개로 설정
print_num = Select(driver.find_element_by_xpath('//*[@id="opt_div"]/div[2]/div[5]/form/select'))
print_num.select_by_value('100')

# 첫번째 다음버튼 선언 (2페이지부터는 '이전'버튼이 생겨 xpath가 바뀜)
first_nextbutton = driver.find_element_by_xpath('/html/body/div[1]/div[1]/div/section[2]/div/div[2]/div[6]/div/div/div[4]/table/tbody/tr/td[2]/a[2]')

p_list = []
count = 0
while True:
    player = driver.find_element_by_id('mytable') # 크롤링 대상인 지표만을 선정
    count += 1
    for p in player.find_elements_by_tag_name('tr'):
        record = p.text
        p_list.append(record.split()) # 한 선수마다 하나의 list로 만들어서 저장
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") # 맨 아래까지 스크롤, 광고로 가려짐을 피하기 위해
    nextbutton = driver.find_element_by_xpath('/html/body/div[1]/div[1]/div/section[2]/div/div[2]/div[6]/div/div/div[4]/table/tbody/tr/td[2]/a[3]')
    time.sleep(2)
    if count == 1: # 첫 번째 페이지일 때
        first_nextbutton.click()
        time.sleep(2)
    else: # 두 번째 페이지 이상일 때
        if nextbutton.text == '마지막':
            print(count, '페이지 수집완료.')
            break
        nextbutton.click()
        time.sleep(2)

C:\Users\Minhyeok\AppData\Local\Temp/ipykernel_8072/2064782435.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver', options=options)
C:\Users\Minhyeok\AppData\Local\Temp/ipykernel_8072/2064782435.py:21: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  optionbutton = driver.find_element_by_xpath('/html/body/div[1]/div[1]/div/section[2]/div/div[2]/div[1]/div/div[8]/button')
C:\Users\Minhyeok\AppData\Local\Temp/ipykernel_8072/2064782435.py:27: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  print_num = Select(driver.find_element_by_xpath('//*[@id="opt_div"]/div[2]/div[5]/form/select'))
C:\Users\Minhyeok\AppData\Local\Temp/ipykernel_8072/2064782435.py:31: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  first_nextbutton = driver.find_element_by_xpath('/h

마지막
다음
다음
마지막
4 페이지 수집완료.


In [6]:
# 10명의 기록마다 컬럼명이 새로 생기므로 이를 없애줌
while True:
    try:
        p_list.remove(['순', '이름', '팀', '정렬', '출장', '완투', '완봉', '선발', '승', '패', '세', '홀드', '이닝', '실점', '자책', '타자', '안타', '2타', '3타', '홈런', '볼넷', '고4', '사구', '삼진', '보크', '폭투', '비율', 'WAR', 'WPA'])
        p_list.remove(['WAR', 'ERA', 'FIP', 'WHIP', 'ERA+', 'FIP+'])
    except:
        print('삭제완료')
        break

삭제완료


In [7]:
# (2021시즌 기록의 경우) ERA+ 지표가 생성되지 않은 선수들에 대해 np.nan값 입력
import numpy as np

for p in p_list:
    if len(p) != 33:
        p.insert(29, np.nan)

In [8]:
# dataframe에 사용할 컬럼명 list로 선언
record_col = ['순', '이름', '팀', '정렬', '출장', '완투', '완봉', '선발', '승', '패', '세', '홀드', '이닝', '실점', '자책', '타자', '안타', '2타', '3타', '홈런', '볼넷', '고4', '사구', '삼진', '보크', '폭투', 'ERA', 'FIP', 'WHIP', 'ERA+', 'FIP+', 'WPA', 'WAR']

In [9]:
import pandas as pd

# 컬럼만 있는 빈 데이터프레임 생성
pitch_df = pd.DataFrame(columns=record_col)
pitch_df

,순,이름,팀,정렬,출장,완투,완봉,선발,승,패,...,삼진,보크,폭투,ERA,FIP,WHIP,ERA+,FIP+,WPA,WAR


In [10]:
# 각 선수들의 기록을 행 형태로 삽입하기 위하여 Series 형태로 변환 뒤 df에 삽입
for p in p_list:
    pitch_df = pitch_df.append(pd.Series(p, index=pitch_df.columns), ignore_index=True)

In [11]:
pitch_df.head()

,순,이름,팀,정렬,출장,완투,완봉,선발,승,패,...,삼진,보크,폭투,ERA,FIP,WHIP,ERA+,FIP+,WPA,WAR
0,1,미란다,21두,7.09,28,1,1,28,14,5,...,225,1,7,2.33,2.67,1.14,185.0,161.1,7.09,3.27
1,2,백정현,21삼,5.28,27,0,0,27,14,5,...,109,0,5,2.63,4.21,1.24,171.9,107.3,5.28,1.78
2,3,고영표,21K,5.23,26,1,1,25,11,6,...,130,0,1,2.92,3.19,1.04,154.0,139.2,5.23,2.27
3,4,켈리,21L,4.96,30,0,0,30,13,8,...,142,1,7,3.15,3.76,1.24,136.8,115.1,4.96,1.61
4,5,원태인,21삼,4.76,26,0,0,26,14,7,...,129,1,5,3.06,3.65,1.25,147.4,123.4,4.76,2.41


In [12]:
pitch_df.shape

(308, 33)

In [13]:
# 시즌과 팀 이름 분리
pitch_df['시즌'] = pitch_df.팀.str[:2]
pitch_df['팀'] = pitch_df.팀.str[2:]
pitch_df.head()

,순,이름,팀,정렬,출장,완투,완봉,선발,승,패,...,보크,폭투,ERA,FIP,WHIP,ERA+,FIP+,WPA,WAR,시즌
0,1,미란다,두,7.09,28,1,1,28,14,5,...,1,7,2.33,2.67,1.14,185.0,161.1,7.09,3.27,21
1,2,백정현,삼,5.28,27,0,0,27,14,5,...,0,5,2.63,4.21,1.24,171.9,107.3,5.28,1.78,21
2,3,고영표,K,5.23,26,1,1,25,11,6,...,0,1,2.92,3.19,1.04,154.0,139.2,5.23,2.27,21
3,4,켈리,L,4.96,30,0,0,30,13,8,...,1,7,3.15,3.76,1.24,136.8,115.1,4.96,1.61,21
4,5,원태인,삼,4.76,26,0,0,26,14,7,...,1,5,3.06,3.65,1.25,147.4,123.4,4.76,2.41,21
